In [32]:
# Import Packages 
import os
import sys
import cv2
import numpy as np
import argparse
import string
import pandas as pd
from scipy import stats
from plotnine import ggplot, aes, geom_line, scale_x_continuous, scale_color_manual

# Import PlantCV library
from plantcv import plantcv as pcv

# Open the leaf disc image, change working directory to output_folder_path
def open_rgb_image(input_folder_path, image, output_folder_path):
    '''
    This function reads an image and changes the working directory to the output folder.

    Parameters:
    input_folder_path (str): path of the input folder containing the image
    image (str): name of the image
    output_folder_path (str): path of the output folder

    Returns:
    tuple: a tuple containing the image, path, and image filename
    '''
    os.chdir(output_folder_path)  # Change the working directory
    
    print(os.getcwd())  # Print the working directory
    print(image) # Print the name of the image file
    
    pcv.params.debug = "none"  # Set PlantCV debug parameter to none to avoid plotting
    filename = input_folder_path + "\\" + image  # Create the filename by concatenating the path and the image name
    img, path, img_filename = pcv.readimage(filename=filename, mode="native")  # Read the image
    
    return img, path, img_filename



def leaf_disc_segment(img, nrow, ncol):
    '''
    This function runs the full analysis of leaf discs from thresholding, masking and segmentation, 
    all the way through outputting a csv file with the final data.
    
    Args:
        img (numpy.ndarray): An input image
        nrow (int): Number of rows for grid-based clustering of objects
        ncol (int): Number of columns for grid-based clustering of objects
    
    Returns:
        output_path (str): Path to the output file containing final data
        imgs (list): A list of output images
        masks (list): A list of output masks
    '''
    
    # Turn off intermediate output printing
    pcv.params.debug = "none" 
    
    # Apply custom thresholding to convert image to black and white
    mask, img = pcv.threshold.custom_range(img=img, lower_thresh=[0,0,0], upper_thresh=[255,255,255], channel='RGB')
    
    # Apply mask to original image to remove background noise
    img = pcv.apply_mask(img=img, mask=mask, mask_color='white')

    # Convert image to grayscale
    a = pcv.rgb2gray_lab(rgb_img=img, channel='l')
    # s = pcv.rgb2gray_hsv(rgb_img=img, channel='s')

    # Convert grayscale image to binary image
    img_binary = pcv.threshold.binary(gray_img=a, threshold=210, max_value=255, object_type='dark')

    # Fill small independent gaps in the image
    ab_fill = pcv.fill(bin_img=img_binary, size=200)

    # Apply the mask to the image to remove remaining background noise
    masked2 = pcv.apply_mask(img=img, mask=ab_fill, mask_color='white')

    # Detect objects in the image
    id_objects, obj_hierarchy = pcv.find_objects(masked2, ab_fill)

    # Set the region of interest to the whole image
    roi1, roi_hierarchy= pcv.roi.rectangle(img=img, x=10, y=10, h=masked2.shape[0]-20, w=masked2.shape[1]-20)

    # Use the detected objects and the ROI to create new objects
    roi_objects, hierarchy3, kept_mask, obj_area = pcv.roi_objects(img=img, roi_contour=roi1, 
                                                                   roi_hierarchy=roi_hierarchy, 
                                                                   object_contour=id_objects, 
                                                                   obj_hierarchy=obj_hierarchy,
                                                                   roi_type='partial')

    # Merge the objects together
    obj, mask = pcv.object_composition(img=img, contours=roi_objects, hierarchy=hierarchy3)

    # Cluster the objects based on the specified row and column numbers
    clusters_i, contours, hierarchies = pcv.cluster_contours(img=img, roi_objects=roi_objects, 
                                                             roi_obj_hierarchy=obj_hierarchy, 
                                                             nrow=nrow, ncol=ncol, 
                                                             show_grid=True)

    # Split the image into multiple images based on the clusters
    output_path, imgs, masks = pcv.cluster_contour_splitimg(img=img, grouped_contour_indexes=clusters_i, 
                                                            contours=contours, hierarchy=hierarchies, 
                                                            outdir=folder_path, file=None, filenames=None)


    return output_path, imgs, masks



def crop_and_split_images(imgs, masks, i):
    """
    This function will run the full analysis of leaf discs from thresholding, masking and segmentation, 
    all the way through outputing a csv file with the final data
    """
    
    # Set debug mode to "plot" for automatic cropping
    pcv.params.debug = "plot"
    crop_img = pcv.auto_crop(imgs[i], masks[i], 20, 20, 'image')

    # Set debug mode back to "none" for the rest of the process
    pcv.params.debug = "none"

    # Convert the RGB image to grayscale
    s = pcv.rgb2gray_hsv(rgb_img=crop_img, channel='s')

    # Convert the grayscale image to binary using thresholding
    thresh = pcv.threshold.binary(gray_img=s, threshold=0, max_value=255, object_type='light')

    # Mask the RGB image using the binary image
    rgb_img = crop_img
    mask = thresh
    masked = cv2.bitwise_and(rgb_img, rgb_img, mask=mask)

    # Split the masked image into its blue, green, and red channels
    b, g, r = cv2.split(masked)

    # Convert the BGR image to LAB
    lab = cv2.cvtColor(masked, cv2.COLOR_BGR2LAB)
    l, m, y = cv2.split(lab)

    # Convert the BGR image to HSV
    hsv = cv2.cvtColor(masked, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    # Create a dictionary of the color channels
    channels = {"b": b, "g": g, "r": r, "l": l, "m": m, "y": y, "h": h, "s": s, "v": v}

    # Define the histogram plot types
    hist_types = {"ALL": ("b", "g", "r", "l", "m", "y", "h", "s", "v"),
                  "RGB": ("b", "g", "r"),
                  "LAB": ("l", "m", "y"),
                  "HSV": ("h", "s", "v")}


    # Create a dictionary of the histograms for each color channel
    histograms = {
        "b": {"label": "blue", "graph_color": "blue",
              "hist": [float(l[0]) for l in cv2.calcHist([channels["b"]], [0], mask, [256], [0, 255])]},
        "g": {"label": "green", "graph_color": "forestgreen",
              "hist": [float(l[0]) for l in cv2.calcHist([channels["g"]], [0], mask, [256], [0, 255])]},
        "r": {"label": "red", "graph_color": "red",
              "hist": [float(l[0]) for l in cv2.calcHist([channels["r"]], [0], mask, [256], [0, 255])]},
        "l": {"label": "lightness", "graph_color": "dimgray",
              "hist": [float(l[0]) for l in cv2.calcHist([channels["l"]], [0], mask, [256], [0, 255])]},
        "m": {"label": "green-magenta", "graph_color": "magenta",
              "hist": [float(l[0]) for l in cv2.calcHist([channels["m"]], [0], mask, [256], [0, 255])]},
        "y": {"label": "blue-yellow", "graph_color": "yellow",
              "hist": [float(l[0]) for l in cv2.calcHist([channels["y"]], [0], mask, [256], [0, 255])]},
        "h": {"label": "hue", "graph_color": "blueviolet",
              "hist": [float(l[0]) for l in cv2.calcHist([channels["h"]], [0], mask, [256], [0, 255])]},
        "s": {"label": "saturation", "graph_color": "cyan",
              "hist": [float(l[0]) for l in cv2.calcHist([channels["s"]], [0], mask, [256], [0, 255])]},
        "v": {"label": "value", "graph_color": "orange",
              "hist": [float(l[0]) for l in cv2.calcHist([channels["v"]], [0], mask, [256], [0, 255])]}
        }

    return(h,histograms, crop_img, rgb_img)



def create_pixel_dataframe(h, histograms, pdf_file_name, crop_img, image, i):
    '''
    This function analyzes a leaf image and outputs a DataFrame with pixel 
    data and a classified image.
    
    Parameters:
        histograms (dict): A dictionary of color channel histograms for the image.
        pdf_file_name (str): The path to the naive Bayes classifier PDF file.
        crop_img (numpy.ndarray): The cropped image to analyze.
        image (str): The name of the full image file.
        i (int): The replicate number.
        
    Returns:
        tuple: A DataFrame with pixel data and a classified image.
    '''

    # Create list of bin labels for 8-bit data
    binval = np.arange(0, 256)
    bin_values = [l for l in binval]

    analysis_image = None

    # Calculate circular statistics for hue channel
    hue_median = np.median(h[np.where(h > 0)]) * 2
    hue_circular_mean = stats.circmean(h[np.where(h > 0)], high=179, low=0) * 2
    hue_circular_std = stats.circstd(h[np.where(h > 0)], high=179, low=0) * 2

    rgb_values = [i for i in range(0, 256)]
    hue_values = [i * 2 + 1 for i in range(0, 180)]
    percent_values = [round((i / 255) * 100, 2) for i in range(0, 256)]
    diverging_values = [i for i in range(-128, 128)]

    # Calculate mean color values for masked image
    masked_b_mean = np.average(histograms["b"]["hist"])
    masked_r_mean = np.average(histograms["r"]["hist"])
    masked_g_mean = np.average(histograms["g"]["hist"])

    # Run naive Bayes classifier on cropped image
    pcv.params.debug = "none"
    mask = pcv.naive_bayes_classifier(rgb_img=crop_img, pdf_file=pdf_file_name)
    
    # Count number of pixels in each class
    sick_plant = np.count_nonzero(mask['D']) # Necrosis pixels
    healthy_plant = np.count_nonzero(mask['H']) # Green pixels
    presymp_plant = np.count_nonzero(mask['PRE']) # Chlorosis pixels
    
    # Calculate percentages of each class
    percent_diseased = sick_plant / (presymp_plant + sick_plant + healthy_plant + 0.00001)
    percent_pre = presymp_plant / (presymp_plant + sick_plant + healthy_plant + 0.00001)
    percent_diseased_full = (sick_plant + presymp_plant) / (presymp_plant + sick_plant + 
                                                            healthy_plant + 0.00001)

    # Print percent diseased for debugging purposes
    print("Percent Disease Area (PDA): " + str(round(percent_diseased * 100,2)))
    
    # Colorize masked image and create DataFrame with pixel data
    pcv.params.debug = "plot"
    classified_img = pcv.visualize.colorize_masks(masks=[mask['H'], mask['D'], mask['PRE'], mask['WB']], 
                                                  colors=['dark green', 'red', 'gold', 'white'])

    # Create metadata dictionary from image names
    meta_data = image.split('_')
    
    # Concat all data into a final dataframe
    dataset = pd.DataFrame({'Full_Image_Name':image,
                            'Experiment':meta_data[0],
                            'Genotype':meta_data[1],
                            'Replicate': i,
                            'HoursPastInfection(hpi)':meta_data[2],
                            'Date':meta_data[3],
                            'Image_filetype':meta_data[4],
                            'PDA': percent_diseased*100,
                            'PDA_full': percent_diseased_full*100, 
                            'PDA_pre': percent_pre*100, 
                            'Hue_mean': hue_circular_mean, 
                            'Hue_med': hue_median,
                            'Hue_std': hue_circular_std,
                            'Red_mean': masked_r_mean,
                            'Green_mean': masked_g_mean,
                            'Blue_mean': masked_b_mean}, 
                             index = [0])

    return(dataset, classified_img)
 

    
def fb_detached_leaf_assay(folder_path, data_name_out, output_folder_path, pdf_file_name, nrow, ncol):
    '''
    This function will run the full analysis of leaf discs from thresholding, masking and 
    segmentation, all the way through outputing a csv file with the final data
    
    Parameters:
        folder_path (string): Full path to the folder containing images to analyze.
        
        data_name_out (string): The name of the final dataset output (without .csv in name)
        
        output_folder_path (string): The full path where the classified and cropped 
                                     images output
        pdf_file_name (string): The full path to the txt file 
                                with the probability density function (PDF)
        nrow (int): number of rows plantcv will use to make grid 
                    (more is better otherwise, two disks can get grouped)
        ncol (int): number of cols plantcv will use to make grid
                    (more is better otherwise, two disks can get grouped)
        
    Returns:
        tuple: A DataFrame with experiment metadata and pixel data.
    '''

    # Create empty dataframe to store final data
    dataset_full = pd.DataFrame() 

    # Loop through each image in folder
    for image in os.listdir(folder_path):

        # Open RGB image and get path to the image
        img, path, img_filename = open_rgb_image(folder_path, image, output_folder_path) 

        # Segment leaf discs and save segmented images and masks
        outpath, imgs, masks = leaf_disc_segment(img, nrow, ncol)

        # Loop through each segmented image
        for i in range(0, len(imgs)):
            
            # Crop and split images into color channels, and create dataframe with pixel data 
            h, histograms, crop_img, rgb_img = crop_and_split_images(imgs, masks, i)
            dataset, classified_img = create_pixel_dataframe(h,histograms, pdf_file_name, crop_img, image, i)
            print(image)

            # Add current dataset to full dataset
            dataset_full = pd.concat([dataset_full,dataset])
            
            # Print RGB and classified image for quality control
            pcv.print_image(img=rgb_img, filename='RGB_Image_' + data_name_out + str(i) + '.jpg')
            pcv.print_image(img=classified_img, filename='Class_Image_' + data_name_out + str(i) + '.jpg')

    # Print final dataset 
    print(dataset_full)
    
    # Save final dataset as a CSV file
    a = len(imgs) # get the number of images processed
    dataset_full.to_csv(data_name_out + str(a) + '.csv', header=True, index=False)

In [34]:
#Input all output and file names here

#folder_path (string): Full path to the folder containing images to analyze.
#data_name_out (string): The name of the final dataset output (without .csv in name)
#output_folder_path (string): WorkDir - The full path where the classified and cropped images output
#pdf_file_name (string): The full path to the txt file with the probability density function (PDF)

#nrow (int): number of rows plantcv will use to make grid 
#                    (more is better otherwise, two disks can get grouped)

#ncol (int): number of cols plantcv will use to make grid
                    #(more is better otherwise, two disks can get grouped)
    
data_name_out = 'LeafDiscAssy_SampleDataSet_Final'
folder_path = "C:\\Users\\[[Your Full Path]]\\Sample_Data\\"
output_folder_path = 'C:\\Users\\[[Your User Name]]\\Downloads\\'
pdf_file_name = 'LeafDiscPDF_LowD.txt' #must be in output_folder_path (WorkDir)

In [35]:
#Run Full Analysis
fb_detached_leaf_assay(folder_path, data_name_out, output_folder_path, pdf_file_name, 15,2) 

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\[[Your Full Path]]\\Sample_Data\\'